In [1]:
## 필요 모듈 Import

# Dash Component
from dash import Dash, html, dcc, State, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

# DB connection, sql
from sqlalchemy import create_engine, text
import pymysql

# DataFrame
import pandas as pd




In [2]:
# DB connection

# DB info
protocol = "mysql+pymysql://"

# user info
user = "bigdata"
password = "Bigdata123!!"
host = "192.168.56.101"
port = "3306"
db_name = "SampleDB"

# Create Engine
engine = create_engine(f"{protocol}{user}:{password}@{host}:{port}/{db_name}")

# Connection
conn = engine.connect()

In [3]:
## Load Data From Database

world = pd.read_sql("SELECT * FROM world;",conn)
continent = pd.read_sql("SELECT DISTINCT continent FROM world;",conn)

con_list = [{"label":list(i)[0], "value":list(i)[0]} for i in continent.values]
print(con_list)

[{'label': 'Asia', 'value': 'Asia'}, {'label': 'Europe', 'value': 'Europe'}, {'label': 'Africa', 'value': 'Africa'}, {'label': 'Americas', 'value': 'Americas'}, {'label': 'Oceania', 'value': 'Oceania'}]


In [4]:
first_row = [html.Div("World's Population, GDP")]

second_row =[
                dbc.Col([html.Label("대륙",
                                    style={"font-size":20,
                                            "text-align": "center"}),
                        dcc.Dropdown(id='dropdown1', options = con_list)
                        ]),
        
                dbc.Col([html.Label("국가",
                                    style={"font-size":20,
                                            "text-align": "center"}),
                        html.Div(id = "output-dropdown1")]),
        
                dbc.Col([html.Label("연도",
                                    style={"font-size":20,
                                            "text-align": "center"}),
                        html.Div(id = "output-dropdown2")]),
            ]


third_row = [
                html.Div(id = "output")
     
            ]



In [5]:
sample_bootstrap = "https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css"
app = Dash(external_stylesheets=[sample_bootstrap])
app.layout = html.Div([
    dbc.Row(first_row, style={"color":"white",
                                                       "font-size":50,
                                                       "background-color":"black",
                                                        "text-align": "center"}),
    dbc.Row(second_row,
            style={"margin-bottom":0, "height":100,"padding-left":30, "padding-right":30,"text-align":"center"}),
    
    
    dbc.Row(third_row)
    
])


In [6]:
@app.callback(
    Output('output-dropdown1','children'),
    Input('dropdown1','value')
)

def update_county_dropdown(continent):
    country = pd.read_sql(f"SELECT distinct country FROM world WHERE continent = '{continent}'", conn)
    country_list = [{"label":list(i)[0], "value":list(i)[0]} for i in country.values]
    return html.Div([
        dcc.Dropdown(
            id = "dropdown2",
            options=country_list)
    ])

@app.callback(
    Output('output-dropdown2','children'),
    Input('dropdown1','value')
)

def update_year_dorpdown(continent):
    year = pd.read_sql(f"SELECT distinct year FROM world WHERE continent = '{continent}'", conn)
    year_list = [{"label":list(i)[0], "value":list(i)[0]} for i in year.values]
    return html.Div([
        dcc.Dropdown(
            id = "dropdown3",
            options=year_list)
    ])


@app.callback(
    Output('output','children'),
    Input('dropdown2','value'),
    Input('dropdown3','value')
)

def update_graph(country,year):
    
    con_data = pd.read_sql(f"SELECT year, sum(pop) as pop, sum(gdpPercap) as gdpPercap FROM world WHERE country = '{country}' GROUP BY year;", conn)
    con_data2 = pd.read_sql(f"SELECT year, gdpPercap FROM world WHERE country = '{country}';", conn)
    colors = ['blue']*con_data['year'].size
    idx = con_data.index[con_data['year'] == year][0]
    colors[idx] = 'crimson'
    fig1 = go.Figure(go.Bar(x=con_data['year'], y=con_data['pop'], marker_color=colors))
    fig2 = go.Figure(go.Line( x=con_data['year'],y=con_data['gdpPercap'], marker_color=colors))
    return dbc.Row([
        dbc.Col([dcc.Graph(id = "bar-graph",figure = fig1)], style={"border-right-style": "solid","border-right-color": "#e6e6e6"}),
        dbc.Col([dcc.Graph(id = "line-graph",figure = fig2)])
        ],
        style={"margin-top":0,"border-bottom-style": "solid","border-bottom-color": "#e6e6e6","border-top-style": "solid","border-top-color": "#e6e6e6"})



In [ ]:
app.run_server(host='192.168.56.101',port=7777)

Dash is running on http://192.168.56.101:7777/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.56.101:7777/ (Press CTRL+C to quit)
192.168.56.1 - - [19/Feb/2022 02:52:41] "GET / HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:41] "GET /_dash-layout HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:41] "GET /_dash-dependencies HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:41] "GET /_favicon.ico?v=2.1.0 HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:42] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:42] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:42] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/charminglee/ana

192.168.56.1 - - [19/Feb/2022 02:52:42] "POST /_dash-update-component HTTP/1.1" 500 -
192.168.56.1 - - [19/Feb/2022 02:52:42] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:42] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/charminglee/ana

192.168.56.1 - - [19/Feb/2022 02:52:42] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/charminglee/ana

192.168.56.1 - - [19/Feb/2022 02:52:42] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/charminglee/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/charminglee/ana

192.168.56.1 - - [19/Feb/2022 02:52:43] "POST /_dash-update-component HTTP/1.1" 500 -
/home/charminglee/anaconda3/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


192.168.56.1 - - [19/Feb/2022 02:52:44] "POST /_dash-update-component HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
192.168.56.1 - - [19/Feb/2022 02:52:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
